> Взят датасет с данными о качестве сна и здоровье

> В структуре следующие поля:
> - ID -- int
> - Gender -- Male / Female
> - Age -- int
> - Occupation -- str (профессия)
> - Sleep Duration -- float (часов)
> - Quality of Sleep -- int (оценка по 10-бальной шкале)
> - Physical Activity -- int (какое-то число)
> - Stress Level -- int (по 10-бальной шкале)
> - BMI Category -- Overweight / Normal / Normal Weight / Obese
> - Blood Pressure -- int/int
> - Hearth Rate -- int
> - Daily Steps -- int
> - Sleep Disorder -- None / Sleep Apnea / Insomnia

> Будем рассматривать длительность сна, качество сна и уровень стресса

## Парсим csv файл

In [20]:
!pip install pandas --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 5.2 MB/s  0:00:025.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pandas]━━━━ 1/2 [pandas]


## Основная часть

In [27]:
import pandas as pd
import genser
# Читаем csv, получаем DataFrame
all_data = pd.read_csv('dataset.csv', encoding='UTF-8')
# Превращаем всё в числа и убираем лишнее
preprocessed_data = Preprocessing(all_data)
# Шкалируем
scaling_data = Scaling(preprocessed_data)
# Нормализуем
norm_data = Normalize(scaling_data)
# Первая корреляционная матрица (сырая)
corr_mx1 = Corr_matrix(norm_data)

# Удаляем выбросы
clean_data = RemoveOutliers(norm_data)
# Удаляем противоречивые и тождественные экземпляры
good_data = RemoveBad(clean_data)
# Новая корреляционная матрица (очищенная)
corr_mx2 = Corr_matrix(good_data)

# Преобразуем в целочисленный вид
int_data = DfToInt(good_data)
# Делаем размерность равной 3
#scaled_mx = PCA(int_data, 3)
scaled_mx, _ = genser.transform_to(int_data.values.tolist(), 3)
scaled_data = pd.DataFrame(scaled_mx)
# Корреляционная матрица
corr_mx3 = Corr_matrix(scaled_data)

print(50*'=')
print("# Рассматриваем зависимость уровня стресса от разных факторов:")
print(50*'=')





Удалено выбросов: 15
Удалено противоречивых экземпляров: 14
# Рассматриваем зависимость уровня стресса от разных факторов:


In [18]:
print("Первая корреляционная матрица (сырая)")
print(50*'=')
print(corr_mx1['Stress Level'])

Первая корреляционная матрица (сырая)
Gender                    -0.396018
Age                       -0.422344
Sleep Duration            -0.811023
Quality of Sleep          -0.898752
Physical Activity Level   -0.034134
Stress Level               1.000000
Heart Rate                 0.670026
Daily Steps                0.186829
BMI                        0.160531
Systolic BP                0.102818
Diastolic BP               0.091811
Name: Stress Level, dtype: float64


In [19]:
print("Новая корреляционная матрица (очищенная)")
print(50*'=')
print(corr_mx2['Stress Level'])

Новая корреляционная матрица (очищенная)
Gender                    -0.484273
Age                       -0.502356
Sleep Duration            -0.794131
Quality of Sleep          -0.894768
Physical Activity Level   -0.130251
Stress Level               1.000000
Heart Rate                 0.647616
Daily Steps                0.083689
BMI                        0.106441
Systolic BP                0.019593
Diastolic BP              -0.005221
Name: Stress Level, dtype: float64


In [20]:
print("Разница между корреляционными матрицами:")
print(50*'=')
print(corr_mx1['Stress Level'] - corr_mx2['Stress Level'])

Разница между корреляционными матрицами:
Gender                     0.088255
Age                        0.080011
Sleep Duration            -0.016892
Quality of Sleep          -0.003984
Physical Activity Level    0.096117
Stress Level               0.000000
Heart Rate                 0.022410
Daily Steps                0.103140
BMI                        0.054089
Systolic BP                0.083226
Diastolic BP               0.097032
Name: Stress Level, dtype: float64


In [28]:
print(int_data)

     Gender  Age  Sleep Duration  Quality of Sleep  Physical Activity Level  \
0         0    0               1                 4                        2   
1         0    0               1                 4                        5   
2         0    0               1                 4                        5   
3         0    0               0                 0                        0   
4         0    0               0                 0                        0   
..      ...  ...             ...               ...                      ...   
369      10   10               9                10                        8   
370      10   10               8                10                        8   
371      10   10               9                10                        8   
372      10   10               9                10                        8   
373      10   10               9                10                        8   

     Stress Level  Heart Rate  Daily Steps  BMI  Sy

In [21]:
print("Корреляционная матрица для другой размерности")
print(50*'=')
print(corr_mx3)

Корреляционная матрица для другой размерности
          0         1         2
0  1.000000  0.189306 -0.275952
1  0.189306  1.000000 -0.483451
2 -0.275952 -0.483451  1.000000


## Предобработка - превращаем нужные данные в числа

In [2]:
def Preprocessing(df):
    df_ = df.copy()

    gender_map = {'Male': 0, 'Female': 1}
    df_['Gender'] = df['Gender'].map(gender_map)

    bmi_map = {'Normal': 0, 'Normal Weight': 1, 'Overweight': 2, 'Obese': 3}
    df_['BMI'] = df['BMI Category'].map(bmi_map)

    disorder_map = {'None': 0, 'Insomnia': 1, 'Sleep Apnea': 2}
    df_['Sleep Disorder'] = df['Sleep Disorder'].map(bmi_map)

    df_[['Systolic BP', 'Diastolic BP']] = df_['Blood Pressure'].str.split('/', expand=True).astype(int)

    cols_to_drop = ['Person ID', 'Occupation', 'BMI Category', 'Blood Pressure', 'Sleep Disorder']
    df_ = df_.drop(columns = [col for col in df_.columns if col in cols_to_drop])

    return df_

## Проводим нормировку по всем признакам
> Приводим все данные к диапазону [0:1] для удобства

In [3]:
def norm_series(series):
    mini = series.min()
    maxi = series.max()
    return (series - mini) / (maxi - mini)

def Normalize(df):
    norm_df = df.copy()
    for col in norm_df.columns:
        norm_df[col] = norm_series(df[col])
    return norm_df


## Шкалирование

In [4]:
from math import sqrt
def scale_series(series):
    n = len(series)
    mu = sum(series) / n
    s = sqrt( sum( (d - mu)**2 for d in series ) / (n-1))
    #print(f"n = {n} ; mu = {mu} ; s = {s}")
    return (series - mu) / s

def Scaling(df):
    scaled_df = df.copy()
    for col in scaled_df:
        scaled_df[col] = scale_series(df[col])
    return scaled_df

## Строим корреляционную матрицу
> Показывает коэффициенты корреляции между множеством переменных

In [5]:
def Corr_matrix(df):
    return df.corr()

## Устраняем выбросы - метод IQR

In [6]:
from math import floor
from math import ceil

# Вычисление квантиля (метод как в numpy - линейная интерполяция)
def quantil(data, q):
    sorted_data = sorted(data)
    n = len(data)
    
    index = 1 + (n - 1) * q
    lower_idx = int(floor(index)) - 1
    upper_idx = int(ceil(index)) - 1
    if lower_idx == upper_idx: return sorted_data[lower_idx]
    
    weight = index - floor(index)
    return sorted_data[lower_idx] * (1 - weight) + sorted_data[upper_idx] * weight

In [7]:
# Получаем "маску" выбросов
def outliers_mask(data, threshold=1.5):
    outliers_mask = pd.DataFrame(False, index=data.index, columns=data.columns)
    for col in data.columns:
        Q1 = quantil(data[col], 0.25)
        Q3 = quantil(data[col], 0.75)
        IQR = Q3 - Q1
        lower = Q1 - threshold * IQR
        upper = Q3 + threshold * IQR
        outliers_mask[col] = (data[col] < lower) | (data[col] > upper)
    return outliers_mask

In [8]:
def RemoveOutliers(df):
    clean_df = df.copy()
    # Маска (1 и 0)
    mask = outliers_mask(clean_df)
    # Считаем выбросы (1 в маске)
    amount = mask.any(axis = 1).sum()
    print(f"Удалено выбросов: {amount}")
    # Если больше 2 выбросов в строчке, удаляем
    to_remove = mask.sum(axis = 1) > 2
    clean_df = clean_df[~to_remove]
    return clean_df

## Удаляем противоречивые экземпляры

In [9]:
def find_bad_elements(df, tol = 0.1):
    difficult = 1 - tol
    # Высокое качество сна + высокий стресс
    high_sleep_quality = df['Quality of Sleep'] > difficult
    high_stress = df['Stress Level'] > difficult
    bad1 = high_sleep_quality & high_stress

    # Высокая активность + низкая продолжительность сна
    high_activity = df['Physical Activity Level'] > difficult
    low_sleep = df['Sleep Duration'] < tol
    bad2 = high_activity & low_sleep

    # Низкий BPM + высокое давление
    low_bpm = df['Heart Rate'] < tol
    high_BP = (df['Systolic BP'] > difficult) | (df['Diastolic BP'] > difficult)
    bad3 = low_bpm & high_BP

    return bad1 | bad2 | bad3

In [10]:
def RemoveBad(df):
    good_df = df.copy()
    mask = find_bad_elements(good_df)
    amount = mask.sum()
    print(f"Удалено противоречивых экземпляров: {amount}")
    good_df = good_df[~mask]
    return good_df

## Приводим в целочисленную форму

In [26]:
def DfToInt(df, multiplier=1e1):
    return (df * multiplier).round().astype(int)

## Меняем размерность - алгоритм PCA

In [12]:
import numpy as np
def flip_vector(U):
    max_abs_cols_U = np.argmax(np.abs(U), axis=0)
    signs = np.sign(U[max_abs_cols_U, range(U.shape[1])])
    return U * signs

In [13]:
# Принимает значения и новую размерность
def PCA(df, m):
    n_samples, n_features = df.shape
    # Центрируем - вычитаем мат. ожидание
    df_centred = df - df.mean(axis=0)
    # Вычисление разложения матрицы
    # U -- левая сингулярная матрица
    # S -- диагональная сингулярная матрица
    # Vt - транспонированная правая сингулярная матрица 
    U, S, Vt = np.linalg.svd(df_centred)
    # Поворот
    U_flipped = flip_vector(U)

    # новый = df * V = U * S * Vt * V = U * S
    df_new = U_flipped[:, :m] * S[:m]
    return df_new

In [25]:
print(all_data)

     Person ID  Gender  Age            Occupation  Sleep Duration  \
0            1    Male   27     Software Engineer             6.1   
1            2    Male   28                Doctor             6.2   
2            3    Male   28                Doctor             6.2   
3            4    Male   28  Sales Representative             5.9   
4            5    Male   28  Sales Representative             5.9   
..         ...     ...  ...                   ...             ...   
369        370  Female   59                 Nurse             8.1   
370        371  Female   59                 Nurse             8.0   
371        372  Female   59                 Nurse             8.1   
372        373  Female   59                 Nurse             8.1   
373        374  Female   59                 Nurse             8.1   

     Quality of Sleep  Physical Activity Level  Stress Level BMI Category  \
0                   6                       42             6   Overweight   
1                